# Section: Encrypted Deep Learning

- Lesson: Reviewing Additive Secret Sharing
- Lesson: Encrypted Subtraction and Public/Scalar Multiplication
- Lesson: Encrypted Computation in PySyft
- Project: Build an Encrypted Database
- Lesson: Encrypted Deep Learning in PyTorch
- Lesson: Encrypted Deep Learning in Keras
- Final Project

# Lesson: Reviewing Additive Secret Sharing

_For more great information about SMPC protocols like this one, visit https://mortendahl.github.io. With permission, Morten's work directly inspired this first teaching segment._

In [1]:
import random
import numpy as np

BASE = 10

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)

def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

def encrypt(secret):
    first  = random.randrange(Q)
    second = random.randrange(Q)
    third  = (secret - first - second) % Q
    return [first, second, third]

def decrypt(sharing):
    return sum(sharing) % Q

def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [2]:
x = encrypt(encode(4.6))
x

[141610880170733542953917375720122144299,
 291432521767647392530811824243977653679,
 154903289011953558656161355597091690847]

In [3]:
y = encrypt(encode(2.3))
y

[81958176519569888481815288811239910873,
 10819446350298131188259295484058582302,
 201195722605299227400370693485297251237]

In [4]:
z = add(x,y)
z

(223569056690303431435732664531362055172,
 8278622642778276648625841947670491568,
 62125666142085538986086771302023197671)

In [5]:
decode(decrypt(z))

6.89999998

# Lesson: Encrypted Subtraction and Public/Scalar Multiplication

In [6]:
field = 23740629843760239486723

In [7]:
x = 5

bob_x_share = 2372385723 # random number
alices_x_share = field - bob_x_share + x

In [8]:
(bob_x_share + alices_x_share) % field

5

In [9]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [10]:
((bob_x_share + alice_x_share) - (bob_y_share + alice_y_share)) % field

4

In [11]:
((bob_x_share - bob_y_share) + (alice_x_share - alice_y_share)) % field

4

In [12]:
bob_x_share + alice_x_share + bob_y_share + alice_y_share

26

In [13]:
bob_z_share = (bob_x_share - bob_y_share)
alice_z_share = (alice_x_share - alice_y_share)

In [14]:
(bob_z_share + alice_z_share) % field

4

In [15]:
def sub(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] - b[i]) % Q)
    return tuple(c)

In [16]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [17]:
bob_x_share + alice_x_share

15

In [18]:
bob_y_share + alice_y_share

11

In [19]:
((bob_y_share * 3) + (alice_y_share * 3)) % field

3

In [20]:
def imul(a, scalar):
    
    # logic here which can multiply by a public scalar
    
    c = list()
    
    for i in range(len(a)):
        c.append((a[i] * scalar) % Q)
        
    return tuple(c)

In [21]:
x = encrypt(encode(5.5))
x

[197186229673132938377494035511567690026,
 116477976001415923082117589062186416392,
 274282485275785632681278930987527382408]

In [22]:
z = imul(x, 3)

In [23]:
decode(decrypt(z))

16.5

# Lesson: Encrypted Computation in PySyft

In [24]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0717 12:29:16.717586 140702262576896 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.
W0717 12:29:16.743697 140702262576896 deprecation_wrapper.py:119] From /home/venktesh/anaconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [25]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [26]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [27]:
x = x.share(bob, alice, crypto_provider=secure_worker)

In [28]:
y = y.share(bob, alice, crypto_provider=secure_worker)

In [29]:
z = x + y
z.get()

tensor([3, 1, 4, 4])

In [30]:
z = x - y
z.get()

tensor([-1,  3,  2,  4])

In [31]:
z = x * y
z.get()

tensor([ 2, -2,  3,  0])

In [32]:
z = x > y
z.get()

tensor([0, 1, 1, 1])

In [33]:
z = x < y
z.get()

tensor([1, 0, 0, 0])

In [34]:
z = x == y
z.get()

tensor([0, 0, 0, 0])

In [35]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [36]:
z = x + y
z.get().float_precision()

tensor([3., 1., 4., 4.])

In [37]:
z = x - y
z.get().float_precision()

tensor([-1.,  3.,  2.,  4.])

In [38]:
z = x * y
z.get().float_precision()

tensor([ 2., -2.,  3.,  0.])

In [39]:
z = x > y
z.get().float_precision()

tensor([0., 1., 1., 1.])

In [40]:
z = x < y
z.get().float_precision()

tensor([1., 0., 0., 0.])

In [41]:
z = x == y
z.get().float_precision()

tensor([0., 0., 0., 0.])

# Project: Build an Encrypted Database

In [42]:
# try this project here!

In [43]:
import string
import torch as th

encoding_dict = {}
decoding_dict = {}

for index,char in enumerate(''+string.ascii_letters+'012456789'+string.punctuation):
    encoding_dict[char] = index
    decoding_dict[index] = char

In [44]:
decoding_dict

{0: 'a',
 1: 'b',
 2: 'c',
 3: 'd',
 4: 'e',
 5: 'f',
 6: 'g',
 7: 'h',
 8: 'i',
 9: 'j',
 10: 'k',
 11: 'l',
 12: 'm',
 13: 'n',
 14: 'o',
 15: 'p',
 16: 'q',
 17: 'r',
 18: 's',
 19: 't',
 20: 'u',
 21: 'v',
 22: 'w',
 23: 'x',
 24: 'y',
 25: 'z',
 26: 'A',
 27: 'B',
 28: 'C',
 29: 'D',
 30: 'E',
 31: 'F',
 32: 'G',
 33: 'H',
 34: 'I',
 35: 'J',
 36: 'K',
 37: 'L',
 38: 'M',
 39: 'N',
 40: 'O',
 41: 'P',
 42: 'Q',
 43: 'R',
 44: 'S',
 45: 'T',
 46: 'U',
 47: 'V',
 48: 'W',
 49: 'X',
 50: 'Y',
 51: 'Z',
 52: '0',
 53: '1',
 54: '2',
 55: '4',
 56: '5',
 57: '6',
 58: '7',
 59: '8',
 60: '9',
 61: '!',
 62: '"',
 63: '#',
 64: '$',
 65: '%',
 66: '&',
 67: "'",
 68: '(',
 69: ')',
 70: '*',
 71: '+',
 72: ',',
 73: '-',
 74: '.',
 75: '/',
 76: ':',
 77: ';',
 78: '<',
 79: '=',
 80: '>',
 81: '?',
 82: '@',
 83: '[',
 84: '\\',
 85: ']',
 86: '^',
 87: '_',
 88: '`',
 89: '{',
 90: '|',
 91: '}',
 92: '~'}

In [45]:
def to_Tensor(str_input, max_length=10):
    str_input = str_input[:max_length]
    if(len(str_input) < max_length):
        str_input = str_input + '.' * (max_length - len(str_input))
    encoded_representation = list()
    for character in str_input:
        encoded_representation.append(encoding_dict[character])
    return th.tensor(encoded_representation).long()

In [46]:
to_Tensor("privacypreserving")

tensor([15, 17,  8, 21,  0,  2, 24, 15, 17,  4])

In [47]:
def tensor_to_string(input_values):
    s = ""
    for value in input_values:
        s += decoding_dict[int(value)]
    return s

In [48]:
def one_hot(index,length):
    one_hot_encoding = th.zeros(length).long()
    one_hot_encoding[index]=1
    return one_hot_encoding

In [49]:
def string_to_one_hot(str_input, max_len=10):

    str_input = str_input[:max_len].lower()

    # pad strings shorter than max len
    if(len(str_input) < max_len):
        str_input = str_input + "0" * (max_len - len(str_input))

    char_vectors = list()
    for char in str_input:
        char_v = one_hot(encoding_dict[char], len(encoding_dict)).unsqueeze(0)
        char_vectors.append(char_v)
        
    return th.cat(char_vectors, dim=0)

In [50]:
string_to_one_hot("privacy")

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [51]:
def strings_equal(str_a, str_b):

    vect = (str_a * str_b).sum(1)
    print(vect)
    x = vect[0]

    for i in range(vect.shape[0] - 1):
        x = x * vect[i + 1]    

    return x

In [52]:
keys=list()
values=list()

In [53]:
    def add_entry(*owners, key, value):
        key = string_to_one_hot(key)
        key = key.share(*owners)
        keys.append(key)
        
        value = to_Tensor(value, max_length=10)
        value = value.share(*owners)
        values.append(value)
        
    def query(*owners, query_str):
        query_matrix = string_to_one_hot(query_str)
        
        query_matrix = query_matrix.share(*owners)

        key_matches = list()
        for key in keys:

            key_match = strings_equal(key, query_matrix)
            key_matches.append(key_match)

        result = values[0] * key_matches[0]

        for i in range(len(values) - 1):
            result += values[i+1] * key_matches[i+1]
            
        result = result.get()

        return tensor_to_string(result).replace(".","")

In [54]:
import syft as sy
hook = sy.TorchHook(th)
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

W0717 12:29:20.288460 140702262576896 hook.py:97] Torch was already hooked... skipping hooking process
W0717 12:29:20.289308 140702262576896 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0717 12:29:20.290157 140702262576896 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0717 12:29:20.290902 140702262576896 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior


In [55]:
add_entry(bob,alice,secure_worker,key="privacy",value="federatedlearning")
add_entry(bob,alice,secure_worker,key="PATE", value="teacherensembles")

query(bob,alice,secure_worker,query_str="PATE")

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:77823682739 -> bob:251967188]
	-> (Wrapper)>[PointerTensor | me:65209218547 -> alice:4162863056]
	-> (Wrapper)>[PointerTensor | me:22482736124 -> secure_worker:25838753630]
	*crypto provider: me*
(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:82399157513 -> bob:66998102188]
	-> (Wrapper)>[PointerTensor | me:60196966699 -> alice:84042057757]
	-> (Wrapper)>[PointerTensor | me:52819294178 -> secure_worker:91747026574]
	*crypto provider: me*


'teacherens'

# Lesson: Encrypted Deep Learning in PyTorch

### Train a Model

In [56]:
import torch as th
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# A Toy Model
model = Net()

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        
train()

tensor(1.6037)
tensor(4.8184)
tensor(19.1086)
tensor(3.3155)
tensor(0.8509)
tensor(0.7845)
tensor(0.7460)
tensor(0.7149)
tensor(0.6874)
tensor(0.6588)
tensor(0.5558)
tensor(0.4598)
tensor(0.3683)
tensor(0.2846)
tensor(0.2133)
tensor(0.1562)
tensor(0.1126)
tensor(0.0815)
tensor(0.0665)
tensor(0.0557)


In [57]:
model(data)

tensor([[-0.0074],
        [ 0.1007],
        [ 1.0541],
        [ 0.8174]], grad_fn=<AddmmBackward>)

## Encrypt the Model and Data

In [58]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [59]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:72389958296 -> alice:22505257925]
 	-> (Wrapper)>[PointerTensor | me:98367154476 -> bob:91002304522]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:96130963917 -> alice:78320861054]
 	-> (Wrapper)>[PointerTensor | me:90227634957 -> bob:84190544111]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:46774834877 -> alice:33300156669]
 	-> (Wrapper)>[PointerTensor | me:95043541315 -> bob:96755911911]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:26527167321 -> alice:37294779876]
 	-> (Wrapper)>[PointerTensor | me:10558893301 -> bob:49472196

In [60]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [61]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:51888022498 -> alice:29099169055]
	-> (Wrapper)>[PointerTensor | me:89675650074 -> bob:12321661533]
	*crypto provider: secure_worker*

In [62]:
encrypted_prediction = encrypted_model(encrypted_data)

In [63]:
encrypted_prediction.get().float_precision()

tensor([[-0.0090],
        [ 0.0990],
        [ 1.0520],
        [ 0.8150]])

# Lesson: Encrypted Deep Learning in Keras


## Step 1: Public Training

Welcome to this tutorial! In the following notebooks you will learn how to provide private predictions. By private predictions, we mean that the data is constantly encrypted throughout the entire process. At no point is the user sharing raw data, only encrypted (that is, secret shared) data. In order to provide these private predictions, Syft Keras uses a library called [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted) under the hood. TF Encrypted combines cutting-edge cryptographic and machine learning techniques, but you don't have to worry about this and can focus on your machine learning application.

You can start serving private predictions with only three steps:
- **Step 1**: train your model with normal Keras.
- **Step 2**: secure and serve your machine learning model (server).
- **Step 3**: query the secured model to receive private predictions (client). 

Alright, let's go through these three steps so you can deploy impactful machine learning services without sacrificing user privacy or model security.

Huge shoutout to the Dropout Labs ([@dropoutlabs](https://twitter.com/dropoutlabs)) and TF Encrypted ([@tf_encrypted](https://twitter.com/tf_encrypted)) teams for their great work which makes this demo possible, especially: Jason Mancuso ([@jvmancuso](https://twitter.com/jvmancuso)), Yann Dupis ([@YannDupis](https://twitter.com/YannDupis)), and Morten Dahl ([@mortendahlcs](https://github.com/mortendahlcs)). 

_Demo Ref: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials_

## Train Your Model in Keras

To use privacy-preserving machine learning techniques for your projects you should not have to learn a new machine learning framework. If you have basic [Keras](https://keras.io/) knowledge, you can start using these techniques with Syft Keras. If you have never used Keras before, you can learn a bit more about it through the [Keras documentation](https://keras.io). 

Before serving private predictions, the first step is to train your model with normal Keras. As an example, we will train a model to classify handwritten digits. To train this model we will use the canonical [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

We borrow [this example](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) from the reference Keras repository.  To train your classification model, you just run the cell below.

In [1]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0717 12:45:29.181986 140666971268864 deprecation.py:506] From /home/venktesh/anaconda3/envs/pysyft/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 51s 844us/sample - loss: 2.2907 - acc: 0.1202 - val_loss: 2.2611 - val_acc: 0.2122
Epoch 2/12
60000/60000 [==============================] - 52s 859us/sample - loss: 2.2429 - acc: 0.2160 - val_loss: 2.2019 - val_acc: 0.4470
Epoch 3/12
60000/60000 [==============================] - 50s 840us/sample - loss: 2.1795 - acc: 0.3204 - val_loss: 2.1210 - val_acc: 0.6285
Epoch 4/12
60000/60000 [==============================] - 49s 822us/sample - loss: 2.0936 - acc: 0.4120 - val_loss: 2.0096 - val_acc: 0.7076
Epoch 5/12
60000/60000 [==============================] - 49s 823us/sample - loss: 1.9768 - acc: 0.4842 - val_loss: 1.8592 - val_acc: 0.7470
Epoch 6/12
60000/60000 [==============================] - 50s 828us/sample - loss: 1.8272 - acc: 0.5423 - val_loss: 1.6705 - val_acc: 0.7741
Epoch 7/12
60000/60000 [=======

In [2]:
## Save your model's weights for future private prediction
model.save('short-conv-mnist.h5')

## Step 2: Load and Serve the Model

Now that you have a trained model with normal Keras, you are ready to serve some private predictions. We can do that using Syft Keras.

To secure and serve this model, we will need three TFEWorkers (servers). This is because TF Encrypted under the hood uses an encryption technique called [multi-party computation (MPC)](https://en.wikipedia.org/wiki/Secure_multi-party_computation). The idea is to split the model weights and input data into shares, then send a share of each value to the different servers. The key property is that if you look at the share on one server, it reveals nothing about the original value (input data or model weights).

We'll define a Syft Keras model like we did in the previous notebook. However, there is a trick: before instantiating this model, we'll run `hook = sy.KerasHook(tf.keras)`. This will add three important new methods to the Keras Sequential class:
 - `share`: will secure your model via secret sharing; by default, it will use the SecureNN protocol from TF Encrypted to secret share your model between each of the three TFEWorkers. Most importantly, this will add the capability of providing predictions on encrypted data.
 - `serve`: this function will launch a serving queue, so that the TFEWorkers can can accept prediction requests on the secured model from external clients.
 - `shutdown_workers`: once you are done providing private predictions, you can shut down your model by running this function. It will direct you to shutdown the server processes manually if you've opted to manually manage each worker.

If you want learn more about MPC, you can read this excellent [blog](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/).

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

W0717 12:55:31.867096 140666971268864 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.
W0717 12:55:31.909756 140666971268864 deprecation_wrapper.py:119] From /home/venktesh/anaconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



## Model

As you can see, we define almost the exact same model as before, except we provide a `batch_input_shape`. This allows TF Encrypted to better optimize the secure computations via predefined tensor shapes. For this MNIST demo, we'll send input data with the shape of (1, 28, 28, 1). 
We also return the logit instead of softmax because this operation is complex to perform using MPC, and we don't need it to serve prediction requests.

In [4]:
num_classes = 10
input_shape = (1, 28, 28, 1)

model = Sequential()


model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 batch_input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, name="logit"))

### Load Pre-trained Weights

With `load_weights` you can easily load the weights you have saved previously after training your model.

In [5]:
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

## Step 3: Setup Your Worker Connectors

Let's now connect to the TFEWorkers (`alice`, `bob`, and `carol`) required by TF Encrypted to perform private predictions. For each TFEWorker, you just have to specify a host.

These workers run a [TensorFlow server](https://www.tensorflow.org/api_docs/python/tf/distribute/Server), which you can either manage manually (`AUTO = False`) or ask the workers to manage for you (`AUTO = True`). If choosing to manually manage them, you will be instructed to execute a terminal command on each worker's host device after calling `model.share()` below.  If all workers are hosted on a single device (e.g. `localhost`), you can choose to have Syft automatically manage the worker's TensorFlow server.

In [6]:
AUTO = False

alice = sy.TFEWorker(host='localhost:4000', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4001', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4002', auto_managed=AUTO)

## Step 4: Split the Model Into Shares

Thanks to `sy.KerasHook(tf.keras)` you can call the `share` method to transform your model into a TF Encrypted Keras model.

If you have asked to manually manage servers above then this step will not complete until they have all been launched. Note that your firewall may ask for Python to accept incoming connection.

In [7]:
model.share(alice, bob, carol)

W0717 12:55:32.579128 140666971268864 deprecation_wrapper.py:119] From /home/venktesh/anaconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/keras/engine/base_layer_utils.py:29: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

I0717 12:55:32.580575 140666971268864 convolutional.py:114] Performing an activation before a pooling layer can result in unnecessary performance loss. Check model definition in case of missed optimization.
W0717 12:55:32.581525 140666971268864 deprecation_wrapper.py:119] From /home/venktesh/anaconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/tensor/native.py:403: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 12:55:32.588148 140666971268864 deprecation_wrapper.py:119] From /home/venktesh/anaconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/tensor/native.py:101: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0

## Step 5: Launch 3 Servers

```
python -m tf_encrypted.player --config /tmp/tfe.config server0
python -m tf_encrypted.player --config /tmp/tfe.config server1
python -m tf_encrypted.player --config /tmp/tfe.config server2```

## Step 6: Serve the Model

Perfect! Now by calling `model.serve`, your model is ready to provide some private predictions. You can set `num_requests` to set a limit on the number of predictions requests served by the model; if not specified then the model will be served until interrupted.

In [8]:
model.serve(num_requests=3)

Served encrypted prediction 1 to client.
Served encrypted prediction 2 to client.
Served encrypted prediction 3 to client.


## Step 7: Run the Client

At this point open up and run the companion notebook: Section 4b - Encrytped Keras Client

## Step 8: Shutdown the Servers

Once your request limit above, the model will no longer be available for serving requests, but it's still secret shared between the three workers above. You can kill the workers by executing the cell below.

**Congratulations** on finishing Part 12: Secure Classification with Syft Keras and TFE!

In [ ]:
model.shutdown_workers()

if not AUTO:
    process_ids = !ps aux | grep '[p]ython -m tf_encrypted.player --config /tmp/tfe.config' | awk '{print $2}'
    for process_id in process_ids:
        !kill {process_id}
        print("Process ID {id} has been killed.".format(id=process_id))

# Keystone Project - Mix and Match What You've Learned

Description: Take two of the concepts you've learned about in this course (Encrypted Computation, Federated Learning, Differential Privacy) and combine them for a use case of your own design. Extra credit if you can get your demo working with [WebSocketWorkers](https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials/advanced/websockets-example-MNIST) instead of VirtualWorkers! Then take your demo or example application, write a blogpost, and share that blogpost in #general-discussion on OpenMined's slack!!!

Inspiration:
- This Course's Code: https://github.com/Udacity/private-ai
- OpenMined's Tutorials: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials
- OpenMined's Blog: https://blog.openmined.org